In [1]:
from selenium import webdriver
from selenium.common import exceptions 
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time

Create the driver

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

# options = webdriver.ChromeOptions()
# options.add_argument('--ignore-certificate-errors')
# options.add_argument('--incognito')
# driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

# driver.get('https://www.stcars.sg/singapore-car/used-cars/toyota-alphard-25a-s-moonroof-a983714/overview')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Driver [C:\Users\andy\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


<ipython-input-2-c585ef9f7201>:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)


Get the URL of the wedsite and create lists to store the values

In [3]:
url = 'https://www.stcars.sg/singapore-car/used-cars/search/pageSize20/page'
value = []
make_model = []
pages = []
con = ['Imported Used', 'Consignment', 'OPC']

In [4]:
driver.get(url + str(1))

In [5]:
col = driver.find_elements_by_xpath('//*[@id="car_listing"]/table[2]/tbody/tr[1]/td/div[1]/a')
col[0].click()

Get the features name

In [6]:
elem_2 = driver.find_elements_by_tag_name('th')

In [7]:
feature = []
for i in range(len(elem_2)):
    feature.append(elem_2[i].text)
feature = feature[:13]

feature

['Price',
 'Registration Date',
 'COE Remaining',
 'Manufactured',
 'Mileage',
 'No. of Owners',
 'Transmission',
 'Engine Capacity',
 'COE',
 'OMV',
 'Paper Value',
 'Depreciation',
 'Type']

Insert extra feature

In [8]:
feature.insert(8, 'fuel_type')

In [9]:
feature

['Price',
 'Registration Date',
 'COE Remaining',
 'Manufactured',
 'Mileage',
 'No. of Owners',
 'Transmission',
 'Engine Capacity',
 'fuel_type',
 'COE',
 'OMV',
 'Paper Value',
 'Depreciation',
 'Type']

Start scraping the website

In [13]:
for page in range(413, 1, -1):      # starting from the last page (if encounter auto shutdown, refer back to the data
    driver.get(url + str(page))     # we save and see what is the latest page and continue from there)
    
    for i in range(1, 21):
        try:
            xpath = '//*[@id="car_listing"]/table[{}]/tbody/tr[1]/td/div[1]/a'.format(i)
            car = driver.find_elements_by_xpath(xpath)
            time.sleep(2)
            car[0].click()
        except:
            continue
        
        img = driver.find_elements_by_tag_name('img')
        if len(img) > 4:
            if img[3].get_attribute('alt') not in con:
                model = driver.find_elements_by_tag_name('h1')
                data = driver.find_elements_by_tag_name('td')
            temp = []
            for item in range(len(data)):
                try:
                    data[item].text != ''
                except exceptions.StaleElementReferenceException as e:
                    driver.back()
                    break
                if data[item].text != '':
                    temp.append(data[item].text)
                temp_1 = temp[:17]
                temp.clear()
                [temp.append(v) for v in temp_1 if v not in temp]
            if len(temp) != 0:
                if '$' in temp[8] or '-' in temp[8]:
                    if temp[8] != 'Petrol-Electric':
                        temp.insert(8, np.nan)
                    
                make_model.append(model[0].text)
                value.append(temp[:14])
                pages.append(page)
            
        time.sleep(5)
        driver.back()
    
    df = pd.DataFrame(value, columns=feature)
    df.insert(0, 'make_model', make_model)
    df.insert(0, 'page', page)
    df.to_csv('Used Car 1.csv', index=False)
        
driver.close()